## Megatron GPT Bootcamp

## Learning objectives

The objective of this boot camp is designed for training very large language models with NVIDIA [Megatron-LM](https://github.com/NVIDIA/Megatron-LM) in a step-wised manner. 

There are two labs, each with a focus point. 

In Lab 1, we will learn the default [Megatron-LM](https://github.com/NVIDIA/Megatron-LM) workflow, highlighting :

 - How to calculate time-to-compute needs for resource planning.
    
 - Understanding Megatron-LM's core engine - Model Parallel Unit(MPU)
 
 - Profiling : core concepts on GPUs performance across multicampus and/or multi-node runs.

In Lab 2, the focus is shifted to the **customization** of [Megatron-LM](https://github.com/NVIDIA/Megatron-LM) workflow. We will walk through and exercise steps to customise the Megatron-LM's workflow in order to address to local language needs.  


* Standard: Python
* Frameworks: PyTorch + Megatron-LM 

It is required to have more than one GPU for this boot camp.

This boot camp is tested on 2 x A100 GPUs with 40G memory. One should also have [NVLink / Switch](https://www.nvidia.com/en-in/data-center/nvlink/).

Start by checking available GPUs in the environment using nvidia-smi 

In [ ]:
!nvidia-smi

Verify you have 2 x A100 GPUs, each with 40G memory. Below is an example of expected outputs: 

            Wed Sep 15 09:14:15 2021       
            +-----------------------------------------------------------------------------+
            | NVIDIA-SMI 460.27.04    Driver Version: 460.27.04    CUDA Version: 11.2     |
            |-------------------------------+----------------------+----------------------+
            | GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
            | Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
            |                               |                      |               MIG M. |
            |===============================+======================+======================|
            |   0  A100-SXM4-40GB      On   | 00000000:07:00.0 Off |                    0 |
            | N/A   24C    P0    57W / 400W |      0MiB / 40536MiB |      4%      Default |
            |                               |                      |             Disabled |
            +-------------------------------+----------------------+----------------------+
            |   1  A100-SXM4-40GB      On   | 00000000:0F:00.0 Off |                    0 |
            | N/A   24C    P0    53W / 400W |      0MiB / 40536MiB |      0%      Default |
            |                               |                      |             Disabled |
            +-------------------------------+----------------------+----------------------+

            +-----------------------------------------------------------------------------+
            | Processes:                                                                  |
            |  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
            |        ID   ID                                                   Usage      |
            |=============================================================================|
            |  No running processes found                                                 |
            +-----------------------------------------------------------------------------+


In [ ]:
# verify nvlink status
!nvidia-smi nvlink --status

Verify NVlink is active. Below is an example of expected outputs: 

        GPU 0: A100-SXM4-40GB (UUID: GPU-2e4d2105-718d-3b94-6f0f-25c148681e83)
             Link 0: 25 GB/s
             Link 1: 25 GB/s
             Link 2: 25 GB/s
             Link 3: 25 GB/s
             Link 4: 25 GB/s
             Link 5: 25 GB/s
             Link 6: 25 GB/s
             Link 7: 25 GB/s
             Link 8: 25 GB/s
             Link 9: 25 GB/s
             Link 10: 25 GB/s
             Link 11: 25 GB/s
        GPU 1: A100-SXM4-40GB (UUID: GPU-49615223-919e-6f9f-ad79-69d86bc1a13b)
             Link 0: 25 GB/s
             Link 1: 25 GB/s
             Link 2: 25 GB/s
             Link 3: 25 GB/s
             Link 4: 25 GB/s
             Link 5: 25 GB/s
             Link 6: 25 GB/s
             Link 7: 25 GB/s
             Link 8: 25 GB/s
             Link 9: 25 GB/s
             Link 10: 25 GB/s
             Link 11: 25 GB/s

In [ ]:
!nsys status -e

Verify profiling capability, the expected output should look something similar to the below:

            Sampling Environment Check
            Linux Kernel Paranoid Level = 2: OK
            Linux Distribution = Ubuntu
            Linux Kernel Version = 4.18.0-305.12.1.el8_4.x86_64: OK
            Linux perf_event_open syscall available: OK
            Sampling trigger event available: OK
            Intel(c) Last Branch Record support: Not Available
            Sampling Environment: OK


To start with, we need to create folders as placeholders for the dataset. We are going to populate these folders later.

In [1]:
import os
os.makedirs('./dataset/EN/32k', exist_ok=True)
os.makedirs('./dataset/EN/50k', exist_ok=True)
os.makedirs('./dataset/SV/32k', exist_ok=True)
os.makedirs('./dataset/SV/56k', exist_ok=True)
os.makedirs('./sv_ckpt/', exist_ok=True)
os.makedirs('./profiles/naive', exist_ok=True)
os.makedirs('./profiles/2ndrun', exist_ok=True)
os.makedirs('./profiles/SV', exist_ok=True)

---
### Tutorial Outline

The following contents will be covered during the Bootcamp :

- **Outlines of Lab 1**
    Megatron 101 in half a day - Please go through the below notebooks sequentially.
    1. [WebCrawling to obtain raw text data](./jupyter_notebook/Megatron-LM/tools/openwebtext/Lab1-1_Website_scraping.ipynb)
    2. [Estimate hours/days needed to execute one end-to-end run per Megatron-LM's configuration](./jupyter_notebook/Lab1-2_EstimateComputeDaysNeeded.ipynb)
    3. [Understanding the core of Megatron-LM - MPU ](./jupyter_notebook/Lab1-3_MegatronFundementals.ipynb)
    4. [About GPT's tokenizer](./jupyter_notebook/Lab1-4_GPT_vocab_merge_files.ipynb)
    5. [Jsonfy and convert to mmap format](./jupyter_notebook/Lab1-5_jsonfy_and_process2mmap.ipynb)
    6. [Megatron runs vs config](./jupyter_notebook/Lab1-6_Observe_GPT_runs_vs_performance.ipynb)


- **Outlines of Lab 2**
    Getting started on training your own language models using Megatron-LM GPT -- Please go through the below notebooks sequentially.
    1. [Fetch and extract Swedish data](./jupyter_notebook/Megatron-LM/tools/openwebtext/Lab2-1_acquiring_data.ipynb)
    2. [Find sentence boundary and deduplicate your data](./jupyter_notebook/Megatron-LM/tools/openwebtext/Lab2-2_SentenceBoundary_and_Deduplicate.ipynb)
    3. [Train your own GPTBPE Tokenizer on your own data ](./jupyter_notebook/Lab2-3_train_own_GPT2BPETokenizer.ipynb)
    4. [Customize preprocess data python script and convert to mmap](./jupyter_notebook/Lab2-4_customize_process2mmap.ipynb)
    5. [The Challenge - Go Big or go home!](./jupyter_notebook/Lab2-5_run_Megatron_with_varying_config.ipynb)



### Tutorial Duration
The lab material will be presented in a 12 hour session. Link to material is available for download at the end of the gpubootcamp. 

### Content Level
Intermediate, Advanced

### Target Audience and Prerequisites
The target audience for this lab is researchers/graduate students and developers who are interested in learning about training very large language models on a super computing cluster.

Basic understanding on Deep learning and Pytorch is required. If you are new to Deep learning and or new to Pytorch, it is recommended to go through the [Distributed_Deep_Learning bootcamp](https://github.com/gpuhackathons-org/gpubootcamp/tree/master/ai/Distributed_Deep_Learning/English/python) and [Pytorch tutorials](https://pytorch.org/tutorials/) as prior.
 
**Disclaimer** : All the results mentioned in the notebooks were tested on a *DGX-2 machine equipped with 2 x A100 GPUs connected via NVLink*. The results would vary when using different hardware and would also depend on the Interconnect bandwidth and the thermal conditions of the machine.

--- 

## Licensing

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).